In [1]:
import os

In [2]:
%pwd

'f:\\Machine Learning Projects (Home)\\Kidney_Disease_Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'f:\\Machine Learning Projects (Home)\\Kidney_Disease_Classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        # config = self.config.data_ingestion
        config = self.config.get('data_ingestion')
        if not config:
            raise KeyError("The 'data_ingestion' section is missing in the config.yaml file.")
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config
    
    # print(self.config)

In [8]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''
        
        try:
            dataset_utl = self.config.source_url
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_utl} into file {zip_download_dir}")
            
            file_id = dataset_utl.split("/")[-2]
            prefix = "https://drive.google.com/uc?export=download&id="
            gdown.download(prefix+file_id, zip_download_dir)
            
            logger.info(f"Downloaded data from {dataset_utl} into file {zip_download_dir}")
        except Exception as e:
            raise e
    
    
    def extract_zip_file(self):
        """
        zip file path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
try:
    config = ConfigurationManager()
    # print(config)
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-10-21 20:58:51,120:INFO:common:yaml file: config\config.yaml loaded successfully]
[2024-10-21 20:58:51,121:INFO:common:yaml file: params.yaml loaded successfully]
[2024-10-21 20:58:51,122:INFO:common:created directory at: artifacts]
[2024-10-21 20:58:51,122:INFO:common:created directory at: artifacts/data_ingestion/]
[2024-10-21 20:58:51,123:INFO:1499887718:Downloading data from https://drive.google.com/file/d/1hKH9IZqk3VObuqDbFDC4-P2g2j-mv8BD/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?export=download&id=1hKH9IZqk3VObuqDbFDC4-P2g2j-mv8BD
From (redirected): https://drive.google.com/uc?export=download&id=1hKH9IZqk3VObuqDbFDC4-P2g2j-mv8BD&confirm=t&uuid=a896e86e-f3ff-425b-be4f-890ce1a309c0
To: f:\Machine Learning Projects (Home)\Kidney_Disease_Classification\artifacts\data_ingestion\data.zip
  3%|▎         | 40.9M/1.63G [00:13<05:38, 4.69MB/s]

KeyboardInterrupt: 

  3%|▎         | 40.9M/1.63G [00:30<05:38, 4.69MB/s]